In [1]:
import os
import pandas as pd
import numpy as np
import datetime

In [2]:
# Изменить на свой путь к данным.
PATH_TO_DATA = ''

# Задача с именами

Есть файл с именами (https://yadi.sk/i/97rbNP2ucGoAKw). Нужно выполнить следующие действия и посчитать результат:

### 1) Отсортировать все имена в лексикографическом порядке  

In [3]:
# Считываем файл с диска, разделяем на отдельные слова, приводим все к нижнему регистру для универсальности
# (на случай если бы слова содержали буквы обоих регистров) и сохраняем в списке.
with open(os.path.join(PATH_TO_DATA,'names.txt'), mode='r') as f:
    names = f.readline().lower().split(sep = ',')
# Удаляем двойные кавычки окружающие слова.
for i in range(len(names)):
    names[i] = names[i].strip('\"')
# Сортируем список.
names.sort()

### 2) Посчитать для каждого имени алфавитную сумму – сумму порядковых номеров букв (MAY: 13 + 1 + 25 = 39)  
### 3) Умножить алфавитную сумму каждого имени на порядковый номер имени в отсортированном списке (индексация начинается с 1).   Например, если MAY находится на 63 месте в списке, то результат для него будет 63 * 39 = 2457.

In [4]:
nums = [0]*(len(names))
for i in range(len(names)):
    s = 0
    for j in names[i]:
        s += ord(j) - 96
    nums[i] = (i+1)*s

### 4) Просуммировать произведения из п. 3 для всех имен из файла и получить число. Это число и будет ответом.

In [5]:
print (sum(nums))

871853874


# Задача с ip адресами

Есть файл со строками (https://yadi.sk/d/u6AOqGqKozTrpw) вида :

< host>\t< ip>\t< page>\n

Нужно вывести 5 айпи-адресов, которые встречаются чаще других.

In [6]:
# В силу привычки и удобста считал файл в датафрейм
hits = pd.read_csv(os.path.join(PATH_TO_DATA,'hits.txt'), sep = '\t', header = None)
# Сделал словарь где ключ - IP, значение - число вхождений этого IP в список.
ip_dict = {}
for i in hits[1]:
    if i not in ip_dict:
        ip_dict[i] = 1
    else:
        ip_dict[i] += 1
# Вывод пяти топовых IP
print (*sorted(ip_dict, key=ip_dict.get, reverse=True)[0:5])

154.157.157.156 82.146.232.163 194.78.107.33 226.247.119.128 21.143.243.182


In [7]:
# Для проверки можно раскоментировать эти строки и посмотреть на число вхождений.
#for w in sorted(ip_dict, key=ip_dict.get, reverse=True):
#    print ('IP адресс',w,'встретился', ip_dict[w],'раз(а)')

# Задача с качеством уроков

В нашей школе ученики и репетиторы занимаются в специальном онлайн-классе, в котором они могут общаться друг с другом, рисовать на доске, переписываться в чате, обмениваться файлами и решать различные задачки. Информацию о каждом уроке мы старательно записываем в базу данных в таблицу lessons. На каждом таком уроке присутствует один репетитор и один ученик (существуют еще тренировочные уроки, у которых единственный участник – репетитор, но для нашей задачи они неинтересны). В конце урока, когда учитель и ученик покидают класс, они могут оценить (а могут и пропустить этот этап) качество пройденного урока по пятибальной шкале. Все оценки мы записываем в таблицу quality, чтобы мы могли узнавать, хорошо ли работает наш сайт.

В этой задаче тебе нужно будет в каждый день найти репетитора, уроки которого получили самую низкую оценку качества, и вывести эту оценку.

В архиве (https://yadi.sk/d/ESH_lU5sZRdKhA) содержится четыре файла с выгрузкой строк из базы.

lessons.txt содержит следующие поля:
id – уникальный идентификатор урока.
event_id – идентификатор, связывающий уроки с файлом participants. У нескольких уроков может быть один и тот же event_id. Подробнее про это будет позже
subject – предмет урока, просто строка
scheduled_time – время начала урока, в формате ГГГГ-ММ-ДД чч:мм:сс (иногда есть еще миллисекунды). Время указано в UTC.

quality.txt содержит следующие поля:
lesson_id – идентификатор урока, указывает на запись в таблице lessons.txt. У нескольких строчек из этого файла может быть один и тот же lesson_id, потому что оценок за урок может быть несколько (например, оба участника, ученик и репетитор могут оценить качество урока)
tech_quality – собственно оценка качества урока. Это число от 1 до 5. Иногда его может не быть, если пользователь не выставил оценку.

users.txt хранит информацию о пользователях и содержит всего два поля:
id – уникальный идентификатор пользователя
role – указывает является ли пользователь учеником (pupil) или учителем (tutor)

Файл participants.txt позволяет связать урок с его участниками. Он содержит следующие поля:
user_id – идентификатор пользователя (указывает на запись в файле users.txt)
event_id – идентификатор, связывающий урок с участником. То есть, чтобы понять, какие пользователи были на уроке Х, нужно найти в файле participants.txt строки, у которых event_id совпадает с event_id урока Х.

Для решения задачки нужно сделать следующее:
1. Найти все уроки по физике (subject=phys).
2. В каждый день (начало и конец дня считается по московскому времени, то есть UTC+3:00) для каждого репетитора посчитать среднюю арифметическую оценку за его уроки (учитывать только уроки из п. 1). То есть, если учитель проводил в этот день три урока по физике, один из них он оценил на 3, а ученик оценил его на 4, второму уроку оценку поставил только ученик, и эта оценка 5, а третий урок вообще никто не оценил, то средняя арифметическая оценка учителя за уроки = (3 + 4 + 5) / 3 = 4.
3. Найти учителя, который в этот день имеет самую низкую среднюю арифметическую оценку за уроки (среди всех учителей, проводивших уроки по физике в этот день).
4. Вывести его в формате "<день> <id учителя> <средняя арифметическая оценка>". Оценку можно округлить с точностью до двух знаков после запятой. То есть как-то так:  
2020-01-11 73c9af08-8581-430c-a590-9888ab36deb3 3.67
2020-01-12 909c2c8e-c054-4e9f-a51a-50bf5660f364 3.25
...
4. Учитывать нужно только тех учителей, за уроки по физике которых в этот день стоит хотя бы одна оценка. Если у нескольких учителей одна и та же самая оценка, можно вывести любого.

## Загрузим данные из результатов запросов в pandas dataframe

In [8]:
PATH_TO_DATA = 'tech_quality'

In [9]:
# Загружаем
lessons = pd.read_csv(os.path.join(PATH_TO_DATA,'lessons.txt'), '|', skiprows = 2, nrows = 378, 
                      names = ['lesson_id','event_id','subject','scheduled_time'], parse_dates = [3])
# Подчищаем лишние пробелы
lessons['lesson_id'] = lessons['lesson_id'].apply(str.strip)
lessons['subject'] = lessons['subject'].apply(str.strip)
# Сдвигаем время на 3 часа для попадания в UTC+03:00
lessons.scheduled_time = lessons.scheduled_time + pd.offsets.Hour(3)
# Получаем дату из времени
lessons['Date'] = lessons.scheduled_time.apply(lambda ts: datetime.date(year = ts.year, month = ts.month, day = ts.day))
lessons.drop(columns = 'scheduled_time', inplace = True)

In [10]:
quality = pd.read_csv(os.path.join(PATH_TO_DATA,'quality.txt'), '|', skiprows = 2, nrows = 365, 
                      names = ['lesson_id','tech_quality'])
quality['lesson_id'] = quality['lesson_id'].apply(str.strip)
# Преобразуем данные в колонке оценки в числовые, с заменой пропусков на NaN
quality['tech_quality'] = pd.to_numeric(quality['tech_quality'], errors='coerce')
# Удалим записи без оценок
quality.dropna(inplace=True)

In [11]:
participants = pd.read_csv(os.path.join(PATH_TO_DATA,'participants.txt'), '|', skiprows = 2, nrows = 743, 
                      names = ['event_id','user_id'])
participants['user_id'] = participants['user_id'].apply(str.strip)

In [12]:
users = pd.read_csv(os.path.join(PATH_TO_DATA,'users.txt'), '|', skiprows = 2, nrows = 743, 
                      names = ['user_id','role'])
users['user_id'] = users['user_id'].apply(str.strip)
users['role'] = users['role'].apply(str.strip)

## Сджоиним данные таким образом что бы решить задачу

In [13]:
# Добавим оценки в таблицу с уроками
lessons_quality = pd.DataFrame.merge(lessons, quality, how='left', on = 'lesson_id')

In [14]:
# Добавим роли участников в таблицу участников
participants_users = pd.DataFrame.merge(participants, users, how='left', on = 'user_id').drop_duplicates()

In [15]:
# Получим список учителей связанных с их ивентами
tutors_list = participants_users[participants_users['role']=='tutor'].drop(columns='role').copy()
tutors_list.columns = ['event_id', 'tutor_id']

In [16]:
# Получим итоговую таблицу достаточную для решения задачи
lessons_quality_tutors = pd.DataFrame.merge(lessons_quality, tutors_list, how='left', on = 'event_id')
lessons_quality_tutors = lessons_quality_tutors.sort_values(by=["Date","lesson_id"])

In [17]:
lessons_quality_tutors.head()

,lesson_id,event_id,subject,Date,tech_quality,tutor_id
71,05a93eb6-a458-44ef-afa6-3a0097319908,48293,phys,2020-01-11,5.0,43efce48-94b2-4412-857f-223d45969008
30,0e30fdbd-2af7-40c7-8f38-336c9d8512f3,38111,phys,2020-01-11,NaN,4df2832a-1d63-4453-9659-43993fc35996
78,0fffa174-fd7c-4266-ba9a-0ad5702853b3,51309,bio,2020-01-11,4.0,7ac76f84-875d-46fb-957a-ba0e390fe539
59,13612a4d-ab26-4c3a-ad28-1b3df22775db,53065,hist,2020-01-11,NaN,f72b84b5-a5a0-4655-8c8b-c84a22735a27
63,1721ee8e-5c4d-4bfc-82e2-42a6a13938cc,51377,it,2020-01-11,NaN,dbfbb7f2-adc9-45e4-bae7-6e0b42b0ecc7


## Оставим только уроки физики

In [18]:
phys = lessons_quality_tutors[lessons_quality_tutors['subject']=='phys'].dropna().drop(columns='event_id').copy()

## Получим средние отметки учителей по дням

In [19]:
means = phys.groupby(['Date', 'tutor_id'], as_index=False).mean().sort_values(by=["Date","tech_quality"])

## Возьмем "худших" учителей дня

In [20]:
mins = means.groupby(['Date'], as_index=False).first()

In [21]:
# Округлим отметки до 2х знаков после запятой
mins.tech_quality = mins.tech_quality.apply(lambda x: np.round(x, 2))

In [22]:
mins

,Date,tutor_id,tech_quality
0,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.43
1,2020-01-12,696c838e-c054-4e9f-a51a-50bf5660f364,4.89
2,2020-01-13,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.00
3,2020-01-14,c6718d0e-976c-4d6c-b0e0-32c770776567,4.00
4,2020-01-15,603b8641-c6f6-4d89-ac89-88e50d45aa0d,5.00
5,2020-01-16,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.00
6,2020-01-17,696c838e-c054-4e9f-a51a-50bf5660f364,4.50
7,2020-01-18,43efce48-94b2-4412-857f-223d45969008,4.25
8,2020-01-19,be676776-8366-4c71-8a35-d58014806eb5,4.50
9,2020-01-20,30a19496-bdaf-461c-abbc-2709ae520201,4.50


In [23]:
for i in range(len(mins)):
    print(mins.iloc[i][0], mins.iloc[i][1], mins.iloc[i][2])

2020-01-11 8fe03f08-8581-430c-a590-9888ab36deb3 4.43
2020-01-12 696c838e-c054-4e9f-a51a-50bf5660f364 4.89
2020-01-13 2fa2ab62-f1b0-4036-872f-bcfd9a8686ff 5.0
2020-01-14 c6718d0e-976c-4d6c-b0e0-32c770776567 4.0
2020-01-15 603b8641-c6f6-4d89-ac89-88e50d45aa0d 5.0
2020-01-16 2fa2ab62-f1b0-4036-872f-bcfd9a8686ff 4.0
2020-01-17 696c838e-c054-4e9f-a51a-50bf5660f364 4.5
2020-01-18 43efce48-94b2-4412-857f-223d45969008 4.25
2020-01-19 be676776-8366-4c71-8a35-d58014806eb5 4.5
2020-01-20 30a19496-bdaf-461c-abbc-2709ae520201 4.5
